In [3]:
# 데이터 준비 
from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000  # 특성으로 사용할 단어의 수
maxlen = 20  # 사용할 텍스트의 길이(가장 빈번한 20 개의 단어만 사용합니다)

# 정수 리스트로 데이터를 로드합니다.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)  # 가장 빈번한 단어 최대 maxlen(20)개만
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train.shape)

(25000, 20)


In [4]:
# 모델 생성
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 가능한 토큰의 개수(1,0000)와 임베딩 차원(8)로 설정 
model.add(Embedding(10000, 8, input_length=maxlen))  # Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됨

model.add(Flatten())  # (samples, maxlen * 8) 크기의 2D 텐서로 펼침

# 분류기를 추가
model.add(Dense(1, activation='sigmoid')) 
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

# 학습
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 75us/step - loss: 0.6698 - acc: 0.6227 - val_loss: 0.6203 - val_acc: 0.6960
Epoch 2/10
20000/20000 [==============================] - 1s 73us/step - loss: 0.5438 - acc: 0.7492 - val_loss: 0.5285 - val_acc: 0.7292
Epoch 3/10
20000/20000 [==============================] -

- 리뷰에서 20개의 단어만 사용한 것 치고는 70%이상의 좋은 결과. 
- 하지만,각 단어를 독립적으로 다루었음. 
- 단어 사이의 관계나 문장의 구조를 고려하지 않았음. 
- 각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 
- 1D 합성곱 층을 추가하는 것이 좋음. 